In [6]:
from quantumml.rest import MWRester
from pymatgen import Structure

In [9]:
with MWRester() as mwr:
    mwr.get_calculation(space_group_number = 10)
    c = mwr.as_pymatgen_struc()
    for e in c:
        assert isinstance(e, Structure) 

# Queries
- **band_gap_range**  
Range of bandgaps in *eV* as a two valued list eg. [1.2, 3.1]  
- **Formation_energy_range**  
Range of formation energies in *meV* as two values list eg. [50, 150]
- **elements**  
List of elements contained in entry eg. ['Na', 'Cl']  
note: this will return all entries that contain Na and Cl as well as any other element 
- **space_group_number**  
integer of space group number eg. 5
- **dimention**  
integer of dimension of material eg. 2
- **crystal_system**  
crystal system as a string eg. 'Tetragonal'  


In [ ]:
help(mwr.get_calculation)

In [ ]:
d = mwr.get_calculation(band_gap_range=[1,1.5], formation_energy_range=[124,150])
for d in mwr.results:
    #print(d.keys())
    if len(d['composition'].split('/')[-2].replace('%',''))< 8:
        print(d['composition'].split('/')[-2].replace('%','')+'\t\t\t BandGap = ' + \
              str(round(d['band_gap'],4))+' eV\t Formation Eneergy = ' + str(d['formation_energy'])+ ' meV')
    elif len(d['composition'].split('/')[-2].replace('%',''))< 16:
        print(d['composition'].split('/')[-2].replace('%','')+'\t\t BandGap = ' + \
              str(round(d['band_gap'],4))+' eV\t Formation Eneergy = ' + str(d['formation_energy'])+ ' meV')
    else:
        print(d['composition'].split('/')[-2].replace('%','')+'\t BandGap = ' + \
              str(round(d['band_gap'],4))+' eV\t Formation Eneergy = ' + str(d['formation_energy'])+ ' meV')


# Write files

In [ ]:
help(mwr.write)
#help(mwr.write_all)

In [ ]:
import os 
os.mkdir('example')
%cd example
mwr.write_all()
%ls
%cd ..

In [ ]:
%cd ~/dev/quantumML
%pwd

# Machine Learning

In [ ]:
#Convert to list of pymatgen structures
print(help(mwr.as_pymatgen_struc()))
c = mwr.as_pymatgen_struc()

In [ ]:
#mwr.get_symmetry_functions_g2(c[0])
#b.get_soap(b.results[0])
!ls

Query materialsproject for all Cd-Te 

In [ ]:
from pymatgen import MPRester
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
import os
from pymatgen.io.vasp import inputs
os.mkdir('mldir')
os.chdir('mldir')
a = MPRester('7JGUQgNZyOTTp8Tc')
entries = a.query('Cd-Te',properties = ['structure','formation_energy_per_atom'])
#entries = a.query('Cd-Te',properties = ['formation_energy_per_atom'])
formation_e_actual=[]
i= 0 
for d in entries:
    d['structure'].to(fmt='poscar',filename=str(i)+'.poscar')
    formation_e_actual.append(d['formation_energy_per_atom'])
    i = i +1 

In [ ]:
!ls

In [ ]:
entries = a.query('Cs',properties = ['structure','formation_energy_per_atom'])
for b in entries:
    b['structure'].to(fmt='poscar',filename=str(i)+'.poscar')
    formation_e_actual.append(b['formation_energy_per_atom'])
    i = i +1 
entries = a.query('Sb',properties = ['structure','formation_energy_per_atom'])
for b in entries:
    b['structure'].to(fmt='poscar',filename=str(i)+'.poscar')
    formation_e_actual.append(b['formation_energy_per_atom'])
    i = i +1 

Retreive pre-trained ML model from materialsweb

In [ ]:
formation_e_predict=[]
model=MWRester().get_KRR('Cd-Te2')
for i in range(0,10):
    #print(i)
    formation_e_predict.append(model.predict(MWRester().get_soap('./'+str(i)+'.poscar',fmt='poscar'))[0])

In [ ]:
type(model)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.scatter(formation_e_actual, formation_e_predict, alpha=0.3)
plt.plot([-0.4, 0.4], [-0.4, 0.4], 'k--')
plt.axis('equal')
plt.show()

In [ ]:
!ls
!mkdir temp

In [ ]:
!cd temp
mwr.prep_ml_formation_energy(mwr.results[-1])

In [ ]:
!ls

In [ ]:
!cd /home/jason/dev

In [ ]:
from dscribe.descriptors import SOAP
from ase.build import molecule
import numpy as np
from ase.io import vasp
structure = vasp.read_vasp('./POSCAR')
#species = ["H","O"]
rcut = 6.0
nmax = 8
lmax = 6

# Setting up the SOAP descriptor
soap = SOAP(
        periodic=True,
        species=np.unique(structure.get_atomic_numbers()),
        rcut=rcut,
        nmax=nmax,
        lmax=lmax,
        rbf='gto',
        sigma=0.125,
        average='inner'
    )

In [ ]:
descriptor = soap.create(structure)

In [ ]:
from sklearn.preprocessing import StandardScaler
descriptor = StandardScaler().fit_transform(descriptor)
#descriptor = scaler.transform(descriptor)

In [ ]:
descriptor

In [ ]:
from ase.io.vasp import write_vasp
write_vasp('POSCAR',water, ignore_constraints = True)